## 이미지 캡셔닝이란?

이미지 캡셔닝은 컴퓨터가 영상을 읽고, 영상을 설명할 수 있는 문장을 생성하는 기술을 의미합니다. 이미지 분류와 같이 라벨 하나로 매칭시키는 것이 아닌, 영상 내 다양한 세부 특징들을 포착해 해석합니다. 이미지 캡셔닝은 사물을 인식하고, 그 관계를 파악하며, 나아가 그것을 자연어인 문장으로 표현하는 것들을 포괄하는 기술입니다. 

### 이미지 캡셔닝을 위한 논문들

해당 문제를 풀기 위해 2014년에 나온 논문인 [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/pdf/1411.4555.pdf)은 영상을 처리하는 CNN 모델과 자연어를 처리하는 RNN 모델을 합쳐 구성하였습니다. 이미지에서 특징을 추출하는 Encoder 역할로 CNN을 두고, 추출된 영상 특징에서 문장을 추출하는 역할로 RNN을 두었습니다. 여기서 발전된 모델로 2015년에 나온 [Show, Attend and Tell: Neural Image Caption Generation With Visual Attention](https://arxiv.org/abs/1502.03044)은 Attention 네트워크를 두었습니다. Attention 네트워크는 RNN과 CNN 사이에 붙어서, 문장을 만들 때 필요한 부분만 주의 집중(Attention)하여 모델을 학습시킵니다. 

두 논문에서 발췌한 구조도 입니다. Attention 네트워크의 유무가 두 논문의 차이입니다.

![Imgur](https://imgur.com/VD2VU03.png)

show, attend and Tell 모델은 아래로 요약할 수 있습니다. 

![Imgur](https://imgur.com/GmXI1zd.png)

### show, Attend and Tell의 핵심, Attention 

영상에 문장을 입히는 것은 영상을 분류하는 것보다 **세부적인 특징**에 훨씬 더 주목해야 합니다. Attention은 영상 속에서 필요한 세부 특성들을 추출하는 데 유용합니다. 각 단어에 맞는 특성들을 영상에서 추출해 줍니다. 

<img src="https://imgur.com/qZxFJBd.png" width="500" > 
***논문 발췌***

a는 영상 속 각 그리드 별 특징 정보, h는 문장을 생성하는 LSTM의 상태 정보를 의미하고, α는 상태 정보 h와 이미지 정보 a간의 관련성을 의미합니다. 해당 논문에서는 α을 산출하는 방법으로 크게 Hard Attention(Stocahstic attention)과 Soft Attention(Deterministic Attention)을 제안합니다. 

<img src="https://imgur.com/CbBEM02.png" width="500">

Hard Attention은 모든 그리드 중 하나를 선택하는 방식으로 동작하고, Soft Attention은 모든 그리드의 Relative Score을 통해 Context Vector을 생성하는 방식으로 동작합니다. 일반적인 자연어처리에서 쓰이는 Attention Network 방식은 Soft Attention입니다. 대표적인 Luong Attention 혹은 Bahdanau Attention 모두 Soft Attention에 속합니다. 구현이 상대적으로 Hard Attention보다는 편리하기 때문에 Soft Attention 방식을 많이들 선호합니다.

### Tell을 담당하는 Decoder, LSTM

문장을 생성하는 역할로 LSTM이 이용됩니다. 매 스텝 별로 순서에 맞게 단어의 확률를 반환합니다. 

<img src="https://imgur.com/owzHmoA.png" width="300" > 

#### show, attend, and Tell 에서의 LSTM

<img src="https://imgur.com/VlzcauE.png" width="500">

본래의 LSTM과 달라진 점이 몇가지 존재합니다. Context Vector가 Attention Network에 의해, 매 단계 별로 영상의 정보가 더해지게 됩니다. 

#### State & Context Vector의 초기화

그리고 LSTM의 State Vector와 Context Vector의 초기화로 단순 영벡터를 쓰는 것이 아니라, 영상의 전체 정보를 요약한 값을 넣어줍니다.

<img src="https://imgur.com/XgUpwcG.png" width="300">

---

## 논문 모델 구현하기

### (1) SHOW 담당, Backbone Network

해당 논문이 나왔던 시점은 2015년도이므로, 당시에 가장 우수한 모델들은 VGG, GoogleNet과 같은 모델들이었습니다. 이제는 훨씬 더 적은 가중치로도 훨씬 더 높은 성능을 보이는 EfficientNet이 있습니다. 보다 빠르고 잘 학습시킬 수 있도록 efficientNet을 이용하도록 하겠습니다.


| | |
|---|---|
|![](https://raw.githubusercontent.com/tensorflow/tpu/master/models/official/efficientnet/g3doc/params.png)|![](https://raw.githubusercontent.com/tensorflow/tpu/master/models/official/efficientnet/g3doc/flops.png) |

케라스의 efficientnet 구현체는 현재 [qubvel/efficientnet](https://github.com/qubvel/efficientnet) 리파짓토리에 구현되어 있습니다.

````shell
pip install efficientnet
````

In [42]:
import efficientnet.tfkeras as efn

backbone = efn.EfficientNetB3(
    input_shape=(224,224,3),
    include_top=False, 
    weights='imagenet')

<tf.Tensor 'top_activation_9/Identity:0' shape=(None, 7, 7, 1536) dtype=float32>

#### Freezing하기

Image Captioning에서 CNN 네트워크는 Pretrained Model을 이용하고, finetune은 하지 않습니다. 그래서 `trainable=False`을 적용합니다.

In [54]:
backbone.trainable = False

#### Squeezing하기

1536은 Attention을 적용시키기에 지나치게 큰 Embedding Size이므로, 임베딩 크기를 줄이도록 유닛 수가 256개인 `Dense`을 둡니다. 

In [64]:
num_embed = 256

squeezed = Dense(num_embed,
                 activation='relu',name='units')(backbone.output)

#### Reshape하기

그리고 출력값의 shape을 (Batch, Height, Width, Channel)에서 (Batch, Height * Width, Channel)로 변경합니다. efficientNet의 최종 출력 벡터인 a는 아래와 같이 구성됩니다. 출력의 grid size는 7x7로 L은 49개가 되고,  각 그리드 별 차원(D)은 256으로 됩니다.

<img src="https://imgur.com/QKtj1oW.png" width="400" >

In [72]:
from tensorflow.keras.layers import Reshape

_, h, w, c = squeezed.get_shape()
embed_image = Reshape((h*w, c))(squeezed)
embed_image

<tf.Tensor 'reshape_7/Identity:0' shape=(None, 49, 256) dtype=float32>

### Attend & Tell 담당 [TODO]

*작성중에 있습니다.*


### Reference

#### Tutorial

* [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/text/image_captioning)

#### Paper
* [A Comprehensive Survey of Deep Learning For Image Captioning](https://arxiv.org/pdf/1810.04020.pdf)
* [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/pdf/1411.4555.pdf)
* [Show, Attend and Tell: Neural Image Caption Generation With Visual Attention](https://arxiv.org/abs/1502.03044)

#### Slideshare
* [Show and Tell: A Neural Image Caption Generator](https://www.slideshare.net/eunjileee/show-attend-and-tell-neural-image-caption-generation-with-visual-attention)